In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR

In [7]:
cd ~

/Users/ae-j


In [8]:
cd Documents/GA/PL/project_group/lancelot_group_project

/Users/ae-j/Documents/GA/PL/project_group/lancelot_group_project


In [9]:
df = pd.read_csv('./data/df_final.csv')

In [32]:
features = df.drop(columns = ['ch_fi_rate_18', 'fi_rate_18', 'cpm_18', 
                       'state_abr', 'state_name', 'fips', 'county'])

X = features
y = df['fi_rate_18']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42)

In [32]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

## Model 1: Linear Regression

In [66]:
LinearRegression().get_params().keys()

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize'])

In [92]:
pipe_ln = Pipeline([
    ('ln', LinearRegression())
])

pipe_ln_params = { 'ln__fit_intercept': [False, True],
                  'ln__copy_X': [False, True],
                  'ln__normalize': [False, True]
}

pipe_ln.fit(Z_train, y_train)
gs_ln = GridSearchCV(pipe_ln, 
                        param_grid = pipe_ln_params, 
                        cv=5)

gs_ln.fit(Z_train, y_train);

In [95]:
print(gs_ln.best_score_)
gs_ln.best_params_

0.9331071494649233


{'ln__copy_X': False, 'ln__fit_intercept': True, 'ln__normalize': False}

In [93]:
ln_preds_train = gs_ln.predict(Z_train)
ln_preds_test = gs_ln.predict(Z_test)

print(f'ln Training RMSE: {mean_squared_error(y_train, ln_preds_train, squared = False)}')
print(f'ln Testing RMSE: {mean_squared_error(y_test, ln_preds_test, squared = False)}')

ln Training RMSE: 0.9242882324129448
ln Testing RMSE: 0.9967000195040713


In [94]:
print(f'Training R2: {pipe_ln.score(Z_train, y_train)}')
print(f'Testing R2: {pipe_ln.score(Z_test, y_test)}')

Training R2: 0.9386462698301442
Testing R2: 0.933007424012739


In [89]:
feature_names = features
coefs_ln = pipe_ln.named_steps['ln'].coef_.flatten()

model_zip_ln = zip(feature_names, coefs_ln)
model_zip_ln_df = pd.DataFrame(model_zip, columns = ['features', 'coefficients'])

model_zip_ln_df.sort_values(by = 'coefficients', ascending = False).head(10)

,features,coefficients
44,percent_disabled,0.767051
16,percent_children_in_poverty,0.552381
5,percent_fair_or_poor_health,0.470569
28,percent_severe_housing_cost_burden,0.334582
20,percent_severe_housing_problems,0.288251
39,percent_below_poverty,0.192885
18,percent_single_parent_households_CHR,0.174937
17,num_households_CHR,0.161619
40,percent_unemployed_CDC,0.151693
43,percent_age_17_and_younger,0.143348


## Model 2: KNN


In [98]:
KNeighborsRegressor().get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [105]:
pipe_knn = Pipeline([
    ('knn', KNeighborsRegressor())
])

pipe_knn_params = { 'knn__n_neighbors': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            }

pipe_knn.fit(Z_train, y_train)
gs_knn = GridSearchCV(pipe_knn, 
                        param_grid = pipe_knn_params, 
                        cv=5)

gs_knn.fit(Z_train, y_train);

In [106]:
print(gs_knn.best_score_)
gs_knn.best_params_

0.851301455655712


{'knn__n_neighbors': 5}

In [108]:
knn_preds_train = gs_knn.predict(Z_train)
knn_preds_test = gs_knn.predict(Z_test)

print(f'knn Training RMSE: {mean_squared_error(y_train, knn_preds_train, squared = False)}')
print(f'knn Testing RMSE: {mean_squared_error(y_test, knn_preds_test, squared = False)}')

knn Training RMSE: 1.1430855549727068
knn Testing RMSE: 1.4479205383412166


In [109]:
print(f'Training R2: {pipe_knn.score(Z_train, y_train)}')
print(f'Testing R2: {pipe_knn.score(Z_test, y_test)}')

Training R2: 0.9061609510423924
Testing R2: 0.8586202527205257


In [110]:
# feature_names = features
# coefs_knn = pipe_knn.named_steps['knn'].coef_.flatten()

# model_zip_knn = zip(feature_names, coefs_knn)
# model_zip_knn_df = pd.DataFrame(model_zip, columns = ['features', 'coefficients'])

# model_zip_knn_df.sort_values(by = 'coefficients', ascending = False).head(10)

AttributeError: 'KNeighborsRegressor' object has no attribute 'coef_'

## Model 3: Decision Tree 

In [111]:
DecisionTreeRegressor().get_params().keys()

dict_keys(['ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])

In [126]:
DecisionTreeRegressor().get_params()

{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [119]:
pipe_dt = Pipeline([
    ('dt', DecisionTreeRegressor())
])

pipe_dt_params = {'dt__max_depth': [5, 10, None], 
                  'dt__max_features': ['auto', 'log2'], 
                'dt__max_depth': [2,4,6,8,10,12]
                 }

pipe_dt.fit(Z_train, y_train)
gs_dt = GridSearchCV(pipe_dt, 
                        param_grid = pipe_dt_params, 
                        cv=5)

gs_dt.fit(Z_train, y_train);

In [120]:
print(gs_dt.best_score_)
gs_dt.best_params_

0.7868272591852264


{'dt__max_depth': 6, 'dt__max_features': 'auto'}

In [121]:
dt_preds_train = gs_dt.predict(Z_train)
dt_preds_test = gs_dt.predict(Z_test)

print(f'dt Training RMSE: {mean_squared_error(y_train, knn_preds_train, squared = False)}')
print(f'dt Testing RMSE: {mean_squared_error(y_test, knn_preds_test, squared = False)}')

dt Training RMSE: 1.1430855549727068
dt Testing RMSE: 1.4479205383412166


In [122]:
print(f'Training R2: {pipe_dt.score(Z_train, y_train)}')
print(f'Testing R2: {pipe_dt.score(Z_test, y_test)}')

Training R2: 1.0
Testing R2: 0.7865900595044211


In [124]:
# feature_names = features
# coefs_dt = pipe_dt.named_steps['dt'].coef_.flatten()

# model_zip_dt = zip(feature_names, coefs_dt)
# model_zip_dt_df = pd.DataFrame(model_zip, columns = ['features', 'coefficients'])

# model_zip_dt_df.sort_values(by = 'coefficients', ascending = False).head(10)

AttributeError: 'DecisionTreeRegressor' object has no attribute 'coef_'